### Project 3 - Group 7 ###

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
import time
from IPython.core.display import clear_output
import requests

API_KEY = '3b7edf89f3e5d5af2746988a93b5c39e'
headers = {'user-agent': 'machinelearning'}
url = 'https://ws.audioscrobbler.com/2.0/'

responses = []
page = 1
total_pages = 99999

while page <= 2: # set the number of pages we want to download. replace with "total_pages" to download everything.

    payload = {'method': 'chart.gettoptracks', 'api_key': API_KEY, 'format': 'json', 'page' : page}

    # print some output so we can see the status
    print("Requesting page {}/{}".format(page, total_pages))
    clear_output(wait = True)

    # make the API call
    response = requests.get(url, headers=headers, params=payload)

    # if we get an error, print the response and halt the loop
    if response.status_code != 200:
        print(response.text)
        break

    # extract pagination info
    page = int(response.json()['tracks']['@attr']['page'])
    total_pages = int(response.json()['tracks']['@attr']['totalPages'])

    # append response
    responses.append(response)

    time.sleep(0.1)

    # increment the page number
    page += 1


frames = [pd.DataFrame(r.json()['tracks']['track']) for r in responses]
df = pd.concat(frames)
df.drop(['duration', 'playcount', 'listeners', 'streamable', 'image'], axis=1, inplace=True)
df

,name,mbid,url,artist
0,As It Was,,https://www.last.fm/music/Harry+Styles/_/As+It...,"{'name': 'Harry Styles', 'mbid': '', 'url': 'h..."
1,N95,,https://www.last.fm/music/Kendrick+Lamar/_/N95,"{'name': 'Kendrick Lamar', 'mbid': '381086ea-f..."
2,United In Grief,,https://www.last.fm/music/Kendrick+Lamar/_/Uni...,"{'name': 'Kendrick Lamar', 'mbid': '381086ea-f..."
3,Worldwide Steppers,,https://www.last.fm/music/Kendrick+Lamar/_/Wor...,"{'name': 'Kendrick Lamar', 'mbid': '381086ea-f..."
4,Die Hard,,https://www.last.fm/music/Kendrick+Lamar/_/Die...,"{'name': 'Kendrick Lamar', 'mbid': '381086ea-f..."
...,...,...,...,...
95,Heartless,a9c00a70-bbf1-4a19-be7e-bdb2d4daf456,https://www.last.fm/music/Kanye+West/_/Heartless,"{'name': 'Kanye West', 'mbid': '164f0d73-1234-..."
96,PUFFIN ON ZOOTIEZ,,https://www.last.fm/music/Future/_/PUFFIN+ON+Z...,"{'name': 'Future', 'mbid': '9f1195cf-9900-45db..."
97,Enemy (with JID) - from the series Arcane Leag...,,https://www.last.fm/music/Imagine+Dragons/_/En...,"{'name': 'Imagine Dragons', 'mbid': '012151a8-..."
98,King Kunta,,https://www.last.fm/music/Kendrick+Lamar/_/Kin...,"{'name': 'Kendrick Lamar', 'mbid': '381086ea-f..."


In [3]:
from bs4 import BeautifulSoup
import json

# Change Max Line Read
pd.options.display.max_colwidth = 1000 

# Getting Artists Names
temp = []
artist_list = []
key = 'name'

# Convert To list
for i in df.artist:
    temp.append(i)

# Get Every Name
for d in temp:
    artist_list.append(d[key])

# Back To Df
df['names'] = artist_list

# Getting Genres
# http://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist=radiohead&track=paranoid+android&api_key=3b7edf89f3e5d5af2746988a93b5c39e
song_list = []
url_list = []

# Links
url = 'https://ws.audioscrobbler.com/2.0/'
API_KEY = '&api_key=3b7edf89f3e5d5af2746988a93b5c39e'
method = '?method=track.gettoptags'

# Convert To List
for i in df.name:
    song_list.append(i)

# Get A Url List
counter = 0
for song in song_list:
    url_change = url + method + '&artist=' + artist_list[counter].replace(" ", "+") + '&track=' + song_list[counter].replace(" ", "+") + API_KEY + '&format=json' 
    url_list.append(url_change)
    counter += 1

# This Will Take Time As It Is Opening Many Pages
responses = []
for url in url_list: 
    response = requests.get(url)
    if response.status_code != 200:
        print(response.text)
        break
    responses.append(response)

In [4]:
# Need to Extract One Genre From Each Page
genres = []
invalids = []
new_genres = []

i = 0
while i <= len(responses) - 1:
    l = responses[i]
    result = json.loads(l.text)
    try:
        if result['toptags']:
            result = result['toptags']
            result = result['tag']
            result = result[0]
            genres.append(result)
            i += 1;
    except:
        invalids.append(i) # Catch Invalids and Iterate Over
        i += 1

for i in genres:
    new_genres.append(i['name'])

In [5]:
df.reset_index(drop=True, inplace=True)
df = df.drop(invalids) # Drop Invalids
df['genres'] = new_genres
df

,name,mbid,url,artist,names,genres
0,As It Was,,https://www.last.fm/music/Harry+Styles/_/As+It+Was,"{'name': 'Harry Styles', 'mbid': '', 'url': 'https://www.last.fm/music/Harry+Styles'}",Harry Styles,indie pop
1,N95,,https://www.last.fm/music/Kendrick+Lamar/_/N95,"{'name': 'Kendrick Lamar', 'mbid': '381086ea-f511-4aba-bdf9-71c753dc5077', 'url': 'https://www.last.fm/music/Kendrick+Lamar'}",Kendrick Lamar,trap
2,United In Grief,,https://www.last.fm/music/Kendrick+Lamar/_/United+In+Grief,"{'name': 'Kendrick Lamar', 'mbid': '381086ea-f511-4aba-bdf9-71c753dc5077', 'url': 'https://www.last.fm/music/Kendrick+Lamar'}",Kendrick Lamar,jazz rap
3,Worldwide Steppers,,https://www.last.fm/music/Kendrick+Lamar/_/Worldwide+Steppers,"{'name': 'Kendrick Lamar', 'mbid': '381086ea-f511-4aba-bdf9-71c753dc5077', 'url': 'https://www.last.fm/music/Kendrick+Lamar'}",Kendrick Lamar,kodak black
4,Die Hard,,https://www.last.fm/music/Kendrick+Lamar/_/Die+Hard,"{'name': 'Kendrick Lamar', 'mbid': '381086ea-f511-4aba-bdf9-71c753dc5077', 'url': 'https://www.last.fm/music/Kendrick+Lamar'}",Kendrick Lamar,rnb
...,...,...,...,...,...,...
145,Heartless,a9c00a70-bbf1-4a19-be7e-bdb2d4daf456,https://www.last.fm/music/Kanye+West/_/Heartless,"{'name': 'Kanye West', 'mbid': '164f0d73-1234-4e2c-8743-d77bf2191051', 'url': 'https://www.last.fm/music/Kanye+West'}",Kanye West,Kanye West
146,PUFFIN ON ZOOTIEZ,,https://www.last.fm/music/Future/_/PUFFIN+ON+ZOOTIEZ,"{'name': 'Future', 'mbid': '9f1195cf-9900-45db-bf9c-21c205672ae0', 'url': 'https://www.last.fm/music/Future'}",Future,trap
147,Enemy (with JID) - from the series Arcane League of Legends,,https://www.last.fm/music/Imagine+Dragons/_/Enemy+(with+JID)+-+from+the+series+Arcane+League+of+Legends,"{'name': 'Imagine Dragons', 'mbid': '012151a8-0f9a-44c9-997f-ebd68b5389f9', 'url': 'https://www.last.fm/music/Imagine+Dragons'}",Imagine Dragons,arcane
148,King Kunta,,https://www.last.fm/music/Kendrick+Lamar/_/King+Kunta,"{'name': 'Kendrick Lamar', 'mbid': '381086ea-f511-4aba-bdf9-71c753dc5077', 'url': 'https://www.last.fm/music/Kendrick+Lamar'}",Kendrick Lamar,Hip-Hop
